Script to exclude subjects based on IQR (statistical outliers).

In [60]:
import os
import pandas as pd
import numpy as np

data_dir = 'data/datasheets'

# Load FreeSurfer output and meta information

Subjects were processed with FreeSurfer's [recon-all pipeline](https://surfer.nmr.mgh.harvard.edu/fswiki/recon-all) (version 7.3.2) and the [ThalamicNuclei pipeline](https://surfer.nmr.mgh.harvard.edu/fswiki/SubregionSegmentation). Volumes in mm<sup>3</sup> were directly extracted from FreeSurfer and loaded here.

In [61]:
# load thalamic nuclei volumes
data = pd.read_csv(os.path.join(data_dir, 'thalamus_volumes_BEST.txt'), index_col=0, sep=' ')

# load aseg
aseg = pd.read_csv(os.path.join(data_dir, 'aseg_BEST7.3.2_volume.txt'), sep='\t', index_col=0)
aseg.index.name='Subject'
tiv = aseg[['EstimatedTotalIntraCranialVol']]
totalGM = aseg[['TotalGrayVol']]

In [62]:
# load covars
meta = pd.read_excel(os.path.join(data_dir, 'BEST_meta_n212.xlsx'), index_col=0)
meta.index.name='Subject'
meta.rename(columns={'Age_at_scan2': 'Age_at_scan'}, inplace=True)

In [63]:
# add tiv and GMV to meta information
meta_tiv = pd.merge(meta, tiv, on="Subject")
meta_tiv = pd.merge(meta_tiv, totalGM, on="Subject")

In [64]:
# merge meta_tiv and data
data_complete_subs = data.merge(meta_tiv, on='Subject', how='inner')

# Quality control

## Quality assessment of cortical reconstruction

For all available subjects, [QA tools by FreeSurfer](https://surfer.nmr.mgh.harvard.edu/fswiki/QATools) was run. Screenshots were visually inspected, and doubtful cases were checked in Freeview. No subject was excluded.

In [65]:
# load csv file containing excluded subjects
excl_recon = pd.read_excel(os.path.join(data_dir, 'QC_BEST_thalamus_nuclei.xlsx'), sheet_name='recon-all')
excl_recon = excl_recon[excl_recon['exclude']==1]

In [66]:
# clean data from excluded subjects
data_complete_subs_sliced = data_complete_subs[~data_complete_subs.index.isin(excl_recon)]

## Quality assessment of the thalamus segmentation

In [67]:
# only select columns containing nuclei volumes and Scanner_ID
data_nuclei = data_complete_subs_sliced['Scanner_ID']
nuclei = data_complete_subs_sliced.filter(like='Left').columns.union(data_complete_subs_sliced.filter(like='Right').columns)
data_nuclei = data_complete_subs_sliced[nuclei].copy()
data_nuclei['Scanner_ID'] = data_complete_subs_sliced['Scanner_ID']

Following [previous literature](https://pubmed.ncbi.nlm.nih.gov/35190533/), the L-Sg, MGN, and LGN were excluded due to poor segmentation quality and/or small size and resulting vulnerability to inaccurate segmentation. 

In [68]:
# drop unrelevant nuclei
data_nuclei = data_nuclei.drop(['Left-LGN','Right-LGN','Left-MGN','Right-MGN','Left-L-Sg','Right-L-Sg'], axis=1)

For each of the remaining 22 nuclei per hemisphere, the statistical outliers are determined by calculating the interquartile range (IQR). Similar to [Weeland et al 2022](https://pubmed.ncbi.nlm.nih.gov/35190533/), subjects with a thalamic nuclei volume of below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR are classified as outliers.

In [69]:
def calculate_iqr(site_number, data):
    '''
    To calculate IQR for each site.
    site_number: [1-4]
    data: df
    '''
    
    # filter data for site
    site_data = data[data['Scanner_ID'] == site_number].copy()
    site_data.drop(['Scanner_ID', 'Left-Whole_thalamus','Right-Whole_thalamus'], axis=1, inplace=True)
    site_data_np = site_data.to_numpy()

    # calculate Q1 and Q3 for each column (i.e., nucleus)
    q1, q3 = np.percentile(site_data_np, [25, 75], axis=0)
    iqr = q3 - q1
    outlier_below = q1 - 1.5 * iqr
    outlier_above = q3 + 1.5 * iqr
    
### get outliers per region
    d = dict()
    lst = []
    keys = site_data.columns

    for roi in range(0, site_data.shape[1]):
        vals = pd.DataFrame(site_data.iloc[:,roi])
    
        for idx, row in vals.iterrows():
            if float(row) < outlier_below[roi] or float(row) > outlier_above[roi]:
                lst.append(idx)
            d[keys[roi]] = lst
            
        lst = []
 
    
### get list of all subjects that have at least one outlier value
    df = pd.DataFrame(list(d.items()), columns = ['ROIs', 'outliers'])

    all_outliers = set() #get rid of duplicates
    for i in d.values():
        for j in i:
            all_outliers.add(j)
          
        
### get amount of outliers per subject
    outlier_lst = [] 
    for i in d.values():
        for j in i:
            outlier_lst.append(j)

    amount_outlier = pd.DataFrame(columns=['Subject_ID','count_outliers'])
    i=0
    for sub in all_outliers:
        count = outlier_lst.count(sub)
        amount_outlier.loc[i,'Subject_ID'] = sub
        amount_outlier.loc[i,'count_outliers'] = count
        i = i+1

    return all_outliers, amount_outlier

In [70]:
# calculate the amount of outliers per site
(all_outliers_s1, amount_outlier_s1) = calculate_iqr(1, data_nuclei)
(all_outliers_s2, amount_outlier_s2) = calculate_iqr(2, data_nuclei)
(all_outliers_s3, amount_outlier_s3) = calculate_iqr(3, data_nuclei)
(all_outliers_s4, amount_outlier_s4) = calculate_iqr(4, data_nuclei)

In [71]:
# concat amount of outliers per subject
outliers_per_sub = pd.concat([amount_outlier_s1, amount_outlier_s2, amount_outlier_s3, amount_outlier_s4], axis=0)
outliers_per_sub = outliers_per_sub.sort_values('count_outliers', ascending=False)
outliers_per_sub.to_csv(os.path.join(data_dir, 'QC_outliers_per_sub_iqrBased.csv'), index=False)

Furthermore, thalamus segmentations by ThalamicNuclei that deviated more than 20% from the native (aseg) FreeSurfer segmentation of the thalamus were also flagged and visually inspected. 

In [72]:
# get thalamus vol based on aseg
aseg_thalamus = aseg[['Left-Thalamus', 'Right-Thalamus']].copy()
aseg_thalamus.rename(columns={'Left-Thalamus':'LThalamus_aseg', 'Right-Thalamus':'RThalamus_aseg'}, inplace=True)

# compare it to thalamus seg volumes
thalseg_thalamus = data_complete_subs_sliced[['Left-Whole_thalamus','Right-Whole_thalamus']].copy()
thalseg_thalamus.rename(columns={'Left-Whole_thalamus':'LThalamus_thalseg', 
                                 'Right-Whole_thalamus':'RThalamus_thalseg'}, inplace=True)

# merge
thalvols = aseg_thalamus.merge(thalseg_thalamus, on='Subject')
thalvols.head()

# calculate ratio of aseg and thalseg
thalvols['ratio_L'] = thalvols['LThalamus_thalseg'] / thalvols['LThalamus_aseg']
thalvols['ratio_R'] = thalvols['RThalamus_thalseg'] / thalvols['RThalamus_aseg']

# mark deviations that are more than 20% 
aseg_deviators = thalvols[(thalvols['ratio_L'] < 0.8) | (thalvols['ratio_L'] > 1.2) | 
                          (thalvols['ratio_R'] < 0.8) | (thalvols['ratio_R'] > 1.2)]
aseg_deviators.to_csv(os.path.join(data_dir, 'QC_aseg_deviators.csv'))

In [73]:
# create a list of subjects to visually inspect
subjects_to_inspect = aseg_deviators.index.to_list() + outliers_per_sub.Subject_ID.to_list()
subjects_to_inspect = set(subjects_to_inspect) #convert to set to have no duplicates
subjects_to_inspect = list(subjects_to_inspect)
subjects_to_inspect = sorted(subjects_to_inspect) #sort alphabetically

file_path = os.path.join(data_dir, 'QC_subjects_to_inspect.txt')
with open(file_path, 'w') as file:
    for element in subjects_to_inspect:
        file.write(str(element) + '\n')

<div class="alert alert-block alert-warning">
<b>Manual intervention:</b> Visually check all outliers saved in <i>QC_outliers_per_sub_iqrBased.csv</i> and aseg deviators saved in <i>QC_aseg_deviators.csv</i> in Freeview and add the ones you consider a segmentation failure below. 
</div>

After inspection, the following criteria were defined for subject exclusion:
- all subjects with more than 5 nuclei that are statistical outliers are excluded
- subjects that did not pass visual quality check are excluded

In [74]:
# exclude > 5 outliers based on IQR
outliers_per_sub_over5 = outliers_per_sub[outliers_per_sub['count_outliers']>= 5]
outliers_per_sub_over5 = outliers_per_sub_over5['Subject_ID'].to_list()

excl = []
for element in outliers_per_sub_over5:
    excl.append(element)

In [75]:
# load notes stating excluded subjects based on visual inspections
further_exclusions = pd.read_excel(os.path.join(data_dir, 'QC_BEST_thalamus_nuclei.xlsx'), sheet_name='thalamus-seg')
further_exclusions = further_exclusions[further_exclusions['exclude']==1]
further_exclusions = further_exclusions.subject_id.to_list()

excl.extend(further_exclusions)

In [76]:
# save final list of excluded subjects
file_name = os.path.join(data_dir, 'QC_excluded_subjects_thalamusseg.txt')
with open(file_name, 'w') as fp:
    for item in excl:
        fp.write("%s\n" % item)
        

In [77]:
# cleaning up and actually excluding flagged subjects
data_nuclei = data_nuclei.drop(['Scanner_ID'], axis=1) 
data_nuclei = data_nuclei.merge(meta_tiv, on='Subject')

data_sliced = data_nuclei[~data_nuclei.index.isin(excl)]
data_sliced.to_csv(os.path.join(data_dir,'thalamus_stats_exclsubs.csv'),index=True)
print("Final amount of subjects:", data_sliced.shape[0])

Final amount of subjects: 175
